In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.datasets import cifar10

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load Dataset

In [2]:
train_xs = np.load('./train_caches.npy')
test_xs = np.load('./test_caches.npy')
(x_train, train_ys), (x_test, test_ys) = cifar10.load_data()
n_classes = 10 
# reshape
test_ys = test_ys.reshape(-1)
train_ys = train_ys.reshape(-1)

# Data provider

In [3]:
import numpy as np
import random

class DataProvider(object):
    def __init__(self, images, labels):
        self.n_sample = len(labels)
        self.queue = list(range(self.n_sample))
        random.shuffle(self.queue)

        self.images = images
        self.labels = labels
        self.epoch_count = 0

    def next_batch(self, batch_size):
        if len(self.queue) < batch_size:
            self.queue = list(range(self.n_sample))
            self.epoch_count += 1
        target_indices = self.queue[:batch_size]
        del self.queue[:batch_size]
        return self.images[target_indices], self.labels[target_indices]

# Modeling

In [4]:
graph = tf.Graph()

with graph.as_default() :
    xs = tf.placeholder(dtype = tf.float32, shape = [None,2048])
    ys = tf.placeholder(dtype = tf.int32 , shape=[None])
    lr = tf.placeholder(dtype = tf.float32, shape = [])
    
    n_units=[1024, 1024]
    layer = xs 
    for ind, units in enumerate(n_units):
        with tf.variable_scope('layer_{}'.format(ind)):
            layer = tf.layers.Dense(units, tf.nn.relu,
                                tf.initializers.he_normal)(layer)
    logits = tf.layers.Dense(n_classes)(layer)

Instructions for updating:
Colocations handled automatically by placer.


# Loss

In [5]:
with graph.as_default() :
    loss = tf.losses.sparse_softmax_cross_entropy(ys, logits)
    loss = tf.identity(loss, 'loss')

Instructions for updating:
Use tf.cast instead.


# Prediction

In [6]:
with graph.as_default() :
    pred = tf.nn.softmax(logits)

## Metric

In [7]:
with graph.as_default() :
    # metric
    logits_cls = tf.argmax(logits, axis=1)
    logits_cls = tf.cast(logits_cls, dtype=tf.int32)
    acc = tf.reduce_mean(tf.cast(tf.equal(ys, logits_cls),tf.float32),
                         name='acc')

## Optimizer

In [8]:
with graph.as_default() :    
    train_op = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9).minimize(loss)

## Open Session

In [10]:
def open_session(graph_def):
    with graph_def.as_default() :    
        config = tf.ConfigProto()
        config.gpu_options.per_process_gpu_memory_fraction = 0.5
        sess = tf.Session()
        init = tf.group(tf.global_variables_initializer(),
                        tf.local_variables_initializer())
        # saver 
        sess.run(init)
        return sess

## Training

In [11]:
def training(sess, graph_def):
    # hparam 
    batch_size = 100
    min_loss = 1000000.0
    learning_rate = 0.001
    max_iter = 10000

    train_loss_ch = []
    train_acc_ch = []
    val_loss_ch = []
    val_acc_ch = []

    dataprovider = DataProvider(images=train_xs, labels=train_ys)
    
    with graph_def.as_default() :
        for i in range(max_iter):
            # extract batch_xs, batch_ys
            batch_xs, batch_ys = dataprovider.next_batch(batch_size)

            # Training 
            sess.run(train_op, {xs: batch_xs, ys: batch_ys, lr: learning_rate})

            # Validation 
            if i % 100 == 0 :
                # Validate validation dataset 
                fetches=[loss, acc]
                val_loss, val_acc = sess.run(fetches, 
                                             {xs: test_xs, ys: test_ys})
                val_loss_ch.append(val_loss)
                val_acc_ch.append(val_acc)

                # Validate train dataset : extract randomly 10000 samples from train dataset 
                batch_xs, batch_ys = dataprovider.next_batch(10000)
                train_loss, train_acc = sess.run(fetches,{xs: batch_xs, ys: batch_ys})
                train_loss_ch.append(train_loss)
                train_acc_ch.append(train_acc)

                print('step : {} train loss : {:.4f} acc : {:.4f} | Val loss : {:.4f} acc : {:.4f}'.\
                format(i, train_loss, train_acc, val_loss, val_acc))

# Generate Multi Session

In [12]:
sessions = [] 
for i in range(2):
    sess = open_session(graph)
    training(sess, graph)
    sessions.append(sess)

step : 0 train loss : 2.3230 acc : 0.0829 | Val loss : 2.3227 acc : 0.0809
step : 100 train loss : 0.9641 acc : 0.8143 | Val loss : 0.9589 acc : 0.8128
step : 200 train loss : 0.5820 acc : 0.8365 | Val loss : 0.5858 acc : 0.8340
step : 300 train loss : 0.4721 acc : 0.8595 | Val loss : 0.4909 acc : 0.8504
step : 400 train loss : 0.4502 acc : 0.8559 | Val loss : 0.4511 acc : 0.8550
step : 500 train loss : 0.4093 acc : 0.8705 | Val loss : 0.4264 acc : 0.8620
step : 600 train loss : 0.3930 acc : 0.8720 | Val loss : 0.4091 acc : 0.8662
step : 700 train loss : 0.3854 acc : 0.8736 | Val loss : 0.3990 acc : 0.8683
step : 800 train loss : 0.3578 acc : 0.8849 | Val loss : 0.3896 acc : 0.8700
step : 900 train loss : 0.3703 acc : 0.8779 | Val loss : 0.3830 acc : 0.8721
step : 1000 train loss : 0.3525 acc : 0.8845 | Val loss : 0.3778 acc : 0.8753
step : 1100 train loss : 0.3459 acc : 0.8842 | Val loss : 0.3707 acc : 0.8753
step : 1200 train loss : 0.3467 acc : 0.8860 | Val loss : 0.3670 acc : 0.877

step : 600 train loss : 0.3889 acc : 0.8729 | Val loss : 0.4060 acc : 0.8673
step : 700 train loss : 0.3827 acc : 0.8730 | Val loss : 0.3957 acc : 0.8700
step : 800 train loss : 0.3593 acc : 0.8828 | Val loss : 0.3884 acc : 0.8710
step : 900 train loss : 0.3664 acc : 0.8751 | Val loss : 0.3793 acc : 0.8748
step : 1000 train loss : 0.3496 acc : 0.8857 | Val loss : 0.3754 acc : 0.8757
step : 1100 train loss : 0.3420 acc : 0.8857 | Val loss : 0.3681 acc : 0.8763
step : 1200 train loss : 0.3441 acc : 0.8859 | Val loss : 0.3643 acc : 0.8808
step : 1300 train loss : 0.3242 acc : 0.8925 | Val loss : 0.3617 acc : 0.8775
step : 1400 train loss : 0.3358 acc : 0.8860 | Val loss : 0.3568 acc : 0.8809
step : 1500 train loss : 0.3236 acc : 0.8916 | Val loss : 0.3565 acc : 0.8801
step : 1600 train loss : 0.3174 acc : 0.8948 | Val loss : 0.3515 acc : 0.8809
step : 1700 train loss : 0.3217 acc : 0.8938 | Val loss : 0.3486 acc : 0.8840
step : 1800 train loss : 0.3027 acc : 0.8989 | Val loss : 0.3476 acc

In [25]:
with graph.as_default():
    pred1 = sessions[0].run(pred, {xs: test_xs})
    pred2 = sessions[1].run(pred, {xs: test_xs})
    
    total_pred = (pred1 + pred2)/2.
    print(total_pred)
    
total_cls = np.argmax(total_pred, axis=1)
ensemble_acc = np.sum(np.equal(total_cls, test_ys)) / len(test_ys)
print(ensemble_acc)

[[4.46635939e-04 1.98418205e-03 1.59415300e-04 ... 2.95155428e-06
  1.13696195e-04 1.03503873e-04]
 [1.12957181e-02 9.77235287e-02 1.27472624e-04 ... 4.07008827e-07
  8.90664101e-01 1.77655878e-04]
 [1.62587059e-03 8.04841635e-04 1.10394240e-06 ... 1.38604196e-06
  9.97544289e-01 2.05346805e-05]
 ...
 [7.24042684e-06 8.29878536e-06 4.49760555e-05 ... 1.60747313e-05
  5.87502700e-06 3.17081285e-05]
 [4.54786390e-01 3.91456664e-01 1.13720410e-01 ... 1.01676420e-03
  1.03244651e-03 6.46431837e-03]
 [3.66316584e-04 9.60541438e-05 7.70299812e-04 ... 9.58935201e-01
  6.16856323e-06 1.44658057e-04]]
0.9015
